# Converting Plot3D to NetworkX Graph Format

This tutorial shows how to convert legacy plot3D block structure to a graph format. 
The figure below shows the numbering of a block.

<div>
<img src="./nodes_edges.jpg" width="800"/>
</div>

You can create a similar blocking structure in the code below


In [ ]:
!pip install plot3d

In [ ]:
import numpy as np
import networkx as nx 
from plot3d import read_plot3D
from plot3d.graph import block_to_graph, get_face_vertex_indices, add_connectivity_to_graph

In [ ]:
IMAX = 4
JMAX = 6
KMAX = 3

A = np.arange(IMAX*JMAX*KMAX).reshape((KMAX,JMAX,IMAX))
A = np.transpose(A,[2,1,0]) 
A_flat = A.flatten(order='F')

In [ ]:
print(A)

The code below shows how Plot3D can create a graph structure representation of a block.
In each of these examples, it's important to step through the code to see how the edges are defined. Use a small block to test. Then you are ready to use larger block sizes. 

In [ ]:
# Create a block
G1 = block_to_graph(IMAX,JMAX,KMAX)
# Create another block but offset indices from the first block
G2 = block_to_graph(IMAX,JMAX,KMAX,IMAX*JMAX*KMAX)

In [ ]:
# Test get face vertex indices 
indices_imin_face = get_face_vertex_indices(0,0,0,0,JMAX,KMAX,(IMAX,JMAX,KMAX))       # Constant IMIN Face
indices_imax_face = get_face_vertex_indices(IMAX,0,0,IMAX,JMAX,KMAX,(IMAX,JMAX,KMAX)) # Constant IMAX Face

indices_jmin_face = get_face_vertex_indices(0,0,0,IMAX,0,KMAX,(IMAX,JMAX,KMAX))       # Constant JMIN Face
indices_jmax_face = get_face_vertex_indices(0,JMAX,0,IMAX,JMAX,KMAX,(IMAX,JMAX,KMAX)) # Constant JMAX Face

indices_kmin_face = get_face_vertex_indices(0,0,0,IMAX,JMAX,0,(IMAX,JMAX,KMAX))       # Constant KMIN Face
indices_kmax_face = get_face_vertex_indices(0,0,KMAX,IMAX,JMAX,KMAX,(IMAX,JMAX,KMAX)) # Constant KMAX Face

indices_jmin_face_reverseI = get_face_vertex_indices(IMAX,0,0,0,0,KMAX,(IMAX,JMAX,KMAX))       # Constant JMIN Face, reversing direction of I

In [ ]:
# Test Connectivity 
G1 = block_to_graph(IMAX,JMAX,KMAX)
G2 = block_to_graph(IMAX,JMAX,KMAX,IMAX*JMAX*KMAX)
G = nx.compose_all([G1,G2])

## Handling Connectivity 

<div>
<img src="./connectivity.jpg" width="800"/>
</div>

Code below shows how plot3d handles connectivity. Connectivity is stored in a dictionary format containing block to block connectivity along with the diagonals of where the connection occurs. 

In [ ]:
# Block 0 and Block 0 share a top face
interblock_k_connectivity = [{
    'block1': 
            {
                'index':0,
                'IMIN':0,'IMAX':IMAX,
                'JMIN':0,'JMAX':JMAX,
                'KMIN':0,'KMAX':0
            },
    'block2': 
            {
                'index':0,
                'IMIN':0,'IMAX':IMAX,
                'JMIN':0,'JMAX':JMAX,
                'KMIN':KMAX,'KMAX':KMAX
            }
    }]

interblock_i_connectivity = [{
    'block1': 
            {
                'index':0,
                'IMIN':0,'IMAX':0,
                'JMIN':0,'JMAX':JMAX,
                'KMIN':0,'KMAX':KMAX
            },
    'block2': 
            {
                'index':0,
                'IMIN':IMAX,'IMAX':IMAX,
                'JMIN':0,'JMAX':JMAX,
                'KMIN':0,'KMAX':KMAX
            }
}]

block_to_block_connectivity = [{
    'block1': 
            {
                'index':0,
                'IMIN':0,'IMAX':0,
                'JMIN':0,'JMAX':JMAX,
                'KMIN':0,'KMAX':KMAX
            },
    'block2': 
            {
                'index':1,
                'IMIN':IMAX,'IMAX':IMAX,
                'JMIN':0,'JMAX':JMAX,
                'KMIN':0,'KMAX':KMAX
            }
}]

`add_connectivity_to_graph` creates the edge to edge connection. For a single block where one face connects to the opposite end kind of like the video game portal. This is a simple edge. 

When you have two different blocks touching, what happens is the nodes are linked then deleted so there are no duplicate nodes. 

In [ ]:
block_sizes = [(IMAX,JMAX,KMAX),(IMAX,JMAX,KMAX)]
G_i_connectivity = add_connectivity_to_graph(G,block_sizes,interblock_i_connectivity)
G_k_connectivity = add_connectivity_to_graph(G,block_sizes,interblock_k_connectivity)


In [ ]:
# Block to Block connectivity 
G1 = block_to_graph(IMAX,JMAX,KMAX)
G2 = block_to_graph(IMAX,JMAX,KMAX,IMAX*JMAX*KMAX)
G = nx.compose_all([G1,G2])
G = add_connectivity_to_graph(G,block_sizes,block_to_block_connectivity)

## Removing Duplicate vertices 

The code in `add_connectivity_to_graph` does not handle the removal of duplication vertices show in the picture below. Once the vertices are removed the edges have to be re-indexed 

<div>
<img src="./reorder_indices.jpg" width="800"/>
</div>

Edges have to be replaced and reindexed.
<div>
<img src="./edges_replaced.jpg" width="800"/>
</div>